In [0]:
import yaml

# Load the configuration
with open("app_config.yaml", "r") as config_file:
    app_config = yaml.safe_load(config_file)

# Access configuration values
endpoint_name = config['env'][0]['value']

In [0]:
%%writefile chatbot_app/main.py
from fastapi import FastAPI
import gradio as gr
import os
from gradio.themes.utils import sizes
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.serving import ChatMessage, ChatMessageRole

app = FastAPI()

# your endpoint will directly be setup with proper permissions when you deploy your app
w = WorkspaceClient()
available_endpoints = [x.name for x in w.serving_endpoints.list()]


def respond(message, history, dropdown):
    if len(message.strip()) == 0:
        return "ERROR the question should not be empty"
    try:
        messages = []
        if history:
            for human, assistant in history:
                messages.append(ChatMessage(content=human, role=ChatMessageRole.USER))
                messages.append(
                    ChatMessage(content=assistant, role=ChatMessageRole.ASSISTANT)
                )
        messages.append(ChatMessage(content=message, role=ChatMessageRole.USER))
        response = w.serving_endpoints.query(
            name=dropdown,
            messages=messages,
            temperature=1.0,
            stream=False,
        )
    except Exception as error:
        return f"ERROR requesting endpoint {dropdown}: {error}"
    return response.choices[0].message.content


theme = gr.themes.Soft(
    text_size=sizes.text_sm,
    radius_size=sizes.radius_sm,
    spacing_size=sizes.spacing_sm,
)

call_app = gr.ChatInterface(
    respond,
    chatbot=gr.Chatbot(
        show_label=False, container=False, show_copy_button=True, bubble_full_width=True
    ),
    textbox=gr.Textbox(placeholder="Ask a question here", container=False, scale=7),
    title="Customer Service Call Transcript Analytics Bot",
    description="This chatbot will help you understand what happens when T-Mobile customers call our customer support. <br>It has access to all our call transcripts to help you discover patterns and build strategies to improve the customer experience.",
    examples=[
        ["How do our customer support agents respond to network issues?"],
        ["What is the most common resolution when customers state they cannot pay their bill?"]
    ],
    cache_examples=False,
    theme=theme,
    retry_btn=None,
    undo_btn=None,
    clear_btn="Clear",
    additional_inputs=gr.Dropdown(
        choices=available_endpoints,
        value=os.environ["MODEL_SERVING_ENDPOINT"],
        label="Serving Endpoint",
    ),
    additional_inputs_accordion="Settings",
)

call_app.queue(default_concurrency_limit=100)
app = gr.mount_gradio_app(app, call_app, path="/")

In [0]:
app_name = "call-transcript-chatbot-app"

#Helper is defined in the _resources/02-lakehouse-app-helpers notebook (temporary helper)
helper = LakehouseAppHelper()
app_details = helper.create(app_name, app_description="Call transcript analysis chatbot app")

In [0]:
helper.add_dependencies(
    app_name=app_name,
    dependencies=[
        {
            "name": "rag-endpoint",
            "serving_endpoint": {
                "name": endpoint_name,
                "permission": "CAN_QUERY",
            },
        }
    ],
    overwrite=False # if False dependencies will be appended to existing ones
)

In [0]:
helper.deploy(app_name, os.path.join(os.getcwd(), 'chatbot_app'))
helper.details(app_name)